In [1]:
# %load 'tools/train_net.py'
#!/usr/bin/env python2

# Copyright (c) 2017-present, Facebook, Inc.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
##############################################################################

"""Train a network with Detectron."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import argparse
import cv2  # NOQA (Must import before importing caffe2 due to bug in cv2)
import logging
import numpy as np
import pprint
import sys

from caffe2.python import workspace

from core.config import assert_and_infer_cfg
from core.config import cfg
from core.config import merge_cfg_from_file
from core.config import merge_cfg_from_list
from core.test_engine import run_inference
from utils.logging import setup_logging
import utils.c2
import utils.train

utils.c2.import_contrib_ops()
utils.c2.import_detectron_ops()
# OpenCL may be enabled by default in OpenCV3; disable it because it's not
# thread safe and causes unwanted GPU memory allocations.
cv2.ocl.setUseOpenCL(False)

Found Detectron ops lib: /usr/local/lib/libcaffe2_detectron_ops_gpu.so


In [4]:
def parse_args():
    parser = argparse.ArgumentParser(
        description='Train a network with Detectron'
    )
    parser.add_argument(
        '--cfg',
        dest='cfg_file',
        help='Config file for training (and optionally testing)',
        default=None,
        type=str
    )
    parser.add_argument(
        '--multi-gpu-testing',
        dest='multi_gpu_testing',
        help='Use cfg.NUM_GPUS GPUs for inference',
        action='store_true'
    )
    parser.add_argument(
        '--skip-test',
        dest='skip_test',
        help='Do not test the final model',
        action='store_true'
    )
    parser.add_argument(
        'opts',
        help='See lib/core/config.py for all options',
        default=None,
        nargs=argparse.REMAINDER
    )
    if len(sys.argv) == 1:
        parser.print_help()
        sys.exit(1)
    return parser.parse_args()

Automatic pdb calling has been turned ON


In [ ]:
def test_model(model_file, multi_gpu_testing, opts=None):
    """Test a model."""
    # Clear memory before inference
    workspace.ResetWorkspace()
    # Run inference
    run_inference(
        model_file, multi_gpu_testing=multi_gpu_testing,
        check_expected_results=True,
    )

In [ ]:
# Initialize C2
workspace.GlobalInit(
    ['caffe2', '--caffe2_log_level=0', '--caffe2_gpu_memory_tracking=1']
    )
# Set up logging and load config options
logger = setup_logging('main')
logging.getLogger('roi_data.loader').setLevel(logging.INFO)
args = parse_args()
logger.info('Called with args:')
logger.info(args)
if args.cfg_file is not None:
    merge_cfg_from_file(args.cfg_file)
if args.opts is not None:
    merge_cfg_from_list(args.opts)
assert_and_infer_cfg()
logger.info('Training with config:')
logger.info(pprint.pformat(cfg))
# Note that while we set the numpy random seed network training will not be
# deterministic in general. There are sources of non-determinism that cannot
# be removed with a reasonble execution-speed tradeoff (such as certain
# non-deterministic cudnn functions).
np.random.seed(cfg.RNG_SEED)
# Execute the training run
checkpoints = utils.train.train_model()
# Test the trained model
if not args.skip_test:
    test_model(checkpoints['final'], args.multi_gpu_testing, args.opts)

In [2]:
from core.config import assert_and_infer_cfg
from core.config import cfg
from core.config import merge_cfg_from_file

merge_cfg_from_file('./configs/caltech/e2e_faster_rcnn_VGG-16-C5_10x_O.yaml')
cfg

{'BBOX_XFORM_CLIP': 4.135166556742356,
 'CLUSTER': {'ON_CLUSTER': False},
 'DATA_LOADER': {'NUM_THREADS': 4},
 'DEDUP_BOXES': 0.0625,
 'DOWNLOAD_CACHE': '/tmp/detectron-download-cache',
 'EPS': 1e-14,
 'EXPECTED_RESULTS': [],
 'EXPECTED_RESULTS_ATOL': 0.005,
 'EXPECTED_RESULTS_EMAIL': '',
 'EXPECTED_RESULTS_RTOL': 0.1,
 'FAST_RCNN': {'MLP_HEAD_DIM': 1024,
  'ROI_BOX_HEAD': 'VGG16.add_VGG16_roi_fc_head',
  'ROI_XFORM_METHOD': 'RoIAlign',
  'ROI_XFORM_RESOLUTION': 14,
  'ROI_XFORM_SAMPLING_RATIO': 0},
 'FPN': {'COARSEST_STRIDE': 32,
  'DIM': 256,
  'EXTRA_CONV_LEVELS': False,
  'FPN_ON': False,
  'MULTILEVEL_ROIS': False,
  'MULTILEVEL_RPN': False,
  'ROI_CANONICAL_LEVEL': 4,
  'ROI_CANONICAL_SCALE': 224,
  'ROI_MAX_LEVEL': 5,
  'ROI_MIN_LEVEL': 2,
  'RPN_ANCHOR_START_SIZE': 32,
  'RPN_ASPECT_RATIOS': (0.5, 1, 2),
  'RPN_MAX_LEVEL': 6,
  'RPN_MIN_LEVEL': 2,
  'ZERO_INIT_LATERAL': False},
 'KRCNN': {'CONV_HEAD_DIM': 256,
  'CONV_HEAD_KERNEL': 3,
  'CONV_INIT': 'GaussianFill',
  'DECONV_DI